# Final Model Architecture(after tuning) along with Training, Inference & Saving of the Models
## This is our saved inference file which contains the architecture of the final **tuned** model and contains the subsequent train, inference and saving processes.
## We have stored our models at both Kaggle and Google Drive whose links are below :
### Kaggle : [https://www.kaggle.com/datasets/supreetsahu/models](https://www.kaggle.com/datasets/supreetsahu/models)
### Google Drive : [https://drive.google.com/drive/folders/1I7XPD0oGj_RubgQ-8Db_OZwYxvNvz1Br?usp=drive_link](https://drive.google.com/drive/folders/1I7XPD0oGj_RubgQ-8Db_OZwYxvNvz1Br?usp=drive_link)
### The Drive folder also contains the original code-files which has the whole training, Tuning using Optuna and inference processes

# **NOTE** : The immediate below section i.e "MODEL INFERENCING PROCESS" is a complete process of training, inference and saving of models. You can see it to know about the final architecture and other parameters.
## **You need not run this section.** 


# 1. MODEL TRAINING & INFERENCING PROCESS

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/under5-gee-new/U5MR_LGBM_TOP_FEATURES_Gain_0.2.csv
/kaggle/input/under5-gee-new/Labels_Under5_with_imputation.parquet.gzip
/kaggle/input/under5-gee-new/U5MR_LGBM_TOP_FEATURES_split.csv
/kaggle/input/under5-gee-new/U5MR_LGBM_TOP_FEATURES_Gain.csv
/kaggle/input/under5-gee-new/Under5_impt_features_LGBM_150.parquet.gzip
/kaggle/input/under5-gee-new/Data_Under5_with_imputation.parquet.gzip
/kaggle/input/under5-gee-new/train_data_Under5_df.parquet.gzip
/kaggle/input/under5-gee-new/U5MR_SHAP_Xtest_new.csv
/kaggle/input/under5-gee-new/U5MR_LGBM_TOP_FEATURES_split_0.2.csv
/kaggle/input/under5-gee-new/Under5_impt_features_LGBM_40.parquet.gzip
/kaggle/input/under5-gee-new/rf_feature_importance_Under5_new.parquet.gzip
/kaggle/input/under5-gee-new/train_labels_Under5_df.parquet.gzip
/kaggle/input/skilled-birth/train_data_Skilled_Birth_df.parquet.gzip
/kaggle/input/skilled-birth/Skilled_birth_impt_features_LGBM_40.parquet.gzip
/kaggle/input/skilled-birth/rf_feature_importance_Skilled_B

In [2]:
pip install --upgrade scipy

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade optuna

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
submission = pd.read_csv('/kaggle/input/test-data-new/sample submission.csv',index_col=0)

## Mean_BMI Model Inferencing
### Tuned at 2101 trees with validation loss = 1.9803820174756033
#### 500 features, bin=255, cv=4 fold

In [6]:
df = pd.read_csv('/kaggle/input/mean-bmi-gee-new/Mean_BMI_SHAP_Xtest_New.csv', index_col=0)
top_features = df['feature'][:500].tolist()

X = pd.read_parquet('/kaggle/input/mean-bmi-gee-new/train_data_Mean_BMI_df.parquet.gzip')[top_features]
y = pd.read_parquet('/kaggle/input/mean-bmi-gee-new/train_labels_Mean_BMI_df.parquet.gzip')
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=23)

# Convert data to LightGBM dataset format
params_d = {'max_bin': 255}
dtrain_data = lgb.Dataset(X, label=y, params=params_d)
# dtest_data = lgb.Dataset(X_valid, label=y_valid, params=params_d)

In [7]:
best_params = {'objective': 'regression',
 'metric': 'rmse',
 'verbosity': -1,
 'boosting': 'gbdt',
 'feature_pre_filter': False,
 'seed': 23,
 'num_threads': -1,
 'device_type': 'gpu',
 'num_gpu': 2,
 'extra_trees': True,
 'learning_rate': 0.014642812105238073,
 'num_leaves': 214,
 'feature_fraction': 0.693111427497064,
 'feature_fraction_bynode': 0.481094980958082,
 'bagging_fraction': 0.9799483675690933,
 'bagging_freq': 912,
 'min_data_in_leaf': 22,
 'min_sum_hessian_in_leaf': 0.08710548089827093,
 'lambda_l1': 3.5675651464836835,
 'lambda_l2': 3.373052826111783,
 'min_gain_to_split': 0.38122614024910395}

In [8]:
verbose_eval=lgb.log_evaluation(period=10, show_stdv=True)
model = lgb.train(params=best_params, train_set=dtrain_data, num_boost_round=2101,
                      valid_sets=[dtrain_data],
                      valid_names=['training'],
                      callbacks=[verbose_eval])

[10]	training's rmse: 3.18182
[20]	training's rmse: 2.95038
[30]	training's rmse: 2.76198
[40]	training's rmse: 2.60717
[50]	training's rmse: 2.48334
[60]	training's rmse: 2.38331
[70]	training's rmse: 2.3025
[80]	training's rmse: 2.23688
[90]	training's rmse: 2.18403
[100]	training's rmse: 2.14205
[110]	training's rmse: 2.10657
[120]	training's rmse: 2.07845
[130]	training's rmse: 2.05482
[140]	training's rmse: 2.03584
[150]	training's rmse: 2.01957
[160]	training's rmse: 2.00399
[170]	training's rmse: 1.99181
[180]	training's rmse: 1.98123
[190]	training's rmse: 1.97147
[200]	training's rmse: 1.96195
[210]	training's rmse: 1.95388
[220]	training's rmse: 1.94636
[230]	training's rmse: 1.93881
[240]	training's rmse: 1.9324
[250]	training's rmse: 1.92595
[260]	training's rmse: 1.91953
[270]	training's rmse: 1.91373
[280]	training's rmse: 1.90801
[290]	training's rmse: 1.90265
[300]	training's rmse: 1.89734
[310]	training's rmse: 1.89202
[320]	training's rmse: 1.88741
[330]	training's rm

In [9]:
# Save the trained model
model.save_model('Mean_BMI.txt')

In [10]:
eval = pd.read_parquet('/kaggle/input/test-data-new/test_data_new.parquet.gzip')[top_features]
print(f'eval shape: {eval.shape}')
y_pred = model.predict(eval)
print(len(y_pred))
label = 'Mean_BMI'
output_df = pd.DataFrame({'DHSID': eval.index, label: y_pred})
output_df.set_index('DHSID', inplace=True)
output_df

eval shape: (14975, 500)
14975


,Mean_BMI
DHSID,
AL200800000003,24.936231
AL200800000005,24.910430
AL200800000007,25.094906
AL200800000008,25.355694
AL200800000009,25.350667
...,...
ZW201500000382,23.931516
ZW201500000383,24.330456
ZW201500000386,24.926128


In [11]:
submission[label] = np.nan
submission.loc[eval.index, label] = output_df.loc[eval.index, label]

In [12]:
listt = [idx for idx in submission.index.tolist() if idx not in output_df.index.tolist()]

In [13]:
submission

,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate
DHSID,,,,,,
AL200800000003,24.936231,20,50,50,50,50
AL200800000005,24.910430,20,50,50,50,50
AL200800000007,25.094906,20,50,50,50,50
AL200800000008,25.355694,20,50,50,50,50
AL200800000009,25.350667,20,50,50,50,50
...,...,...,...,...,...,...
ZW201500000382,23.931516,20,50,50,50,50
ZW201500000383,24.330456,20,50,50,50,50
ZW201500000386,24.926128,20,50,50,50,50


## Median_BMI Model Inferencing
### Tuned at 1795 trees with validation loss = 2.1232553617695564
#### 300 features, bin=255, cv=3 fold

In [14]:
df = pd.read_csv('/kaggle/input/median-bmi-gee-new/Median_BMI_SHAP_Xtest_New.csv', index_col=0)
top_features = df['feature'][:300].tolist()

X = pd.read_parquet('/kaggle/input/median-bmi-gee-new/train_data_Median_BMI_df.parquet.gzip')[top_features]
y = pd.read_parquet('/kaggle/input/median-bmi-gee-new/train_labels_Median_BMI_df.parquet.gzip')
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=23)

# Convert data to LightGBM dataset format
params_d = {'max_bin': 255}
dtrain_data = lgb.Dataset(X, label=y, params=params_d)
# dtest_data = lgb.Dataset(X_valid, label=y_valid, params=params_d)

In [15]:
best_params = {'objective': 'regression',
 'metric': 'rmse',
 'verbosity': -1,
 'boosting': 'gbdt',
 'feature_pre_filter': False,
 'seed': 23,
 'num_threads': -1,
 'device_type': 'gpu',
 'num_gpu': 2,
 'extra_trees': True,
 'learning_rate': 0.010346858765034533,
 'num_leaves': 488,
 'feature_fraction': 0.7006885477994251,
 'feature_fraction_bynode': 0.5561717266207711,
 'bagging_fraction': 0.9946320056796824,
 'bagging_freq': 302,
 'min_data_in_leaf': 84,
 'min_sum_hessian_in_leaf': 0.08413227440053907,
 'lambda_l1': 5.128639551774848,
 'lambda_l2': 5.268197238314045,
 'min_gain_to_split': 0.024156082841999823}

In [16]:
verbose_eval=lgb.log_evaluation(period=10, show_stdv=True)
model = lgb.train(params=best_params, train_set=dtrain_data, num_boost_round=1795,
                      valid_sets=[dtrain_data],
                      valid_names=['training'],
                      callbacks=[verbose_eval])

[10]	training's rmse: 3.34565
[20]	training's rmse: 3.17616
[30]	training's rmse: 3.02837
[40]	training's rmse: 2.90096
[50]	training's rmse: 2.79088
[60]	training's rmse: 2.6961
[70]	training's rmse: 2.61423
[80]	training's rmse: 2.54377
[90]	training's rmse: 2.48377
[100]	training's rmse: 2.43262
[110]	training's rmse: 2.38836
[120]	training's rmse: 2.35066
[130]	training's rmse: 2.31793
[140]	training's rmse: 2.2895
[150]	training's rmse: 2.26517
[160]	training's rmse: 2.24421
[170]	training's rmse: 2.22636
[180]	training's rmse: 2.21077
[190]	training's rmse: 2.1969
[200]	training's rmse: 2.18477
[210]	training's rmse: 2.17412
[220]	training's rmse: 2.16354
[230]	training's rmse: 2.15427
[240]	training's rmse: 2.14629
[250]	training's rmse: 2.1387
[260]	training's rmse: 2.13229
[270]	training's rmse: 2.12622
[280]	training's rmse: 2.12004
[290]	training's rmse: 2.11438
[300]	training's rmse: 2.10903
[310]	training's rmse: 2.10413
[320]	training's rmse: 2.09967
[330]	training's rmse

In [17]:
# Save the trained model
model.save_model('Median_BMI.txt')

In [18]:
eval = pd.read_parquet('/kaggle/input/test-data-new/test_data_new.parquet.gzip')[top_features]
print(f'eval shape: {eval.shape}')
y_pred = model.predict(eval)
print(len(y_pred))
label = 'Median_BMI'
output_df = pd.DataFrame({'DHSID': eval.index, label: y_pred})
output_df.set_index('DHSID', inplace=True)
output_df

eval shape: (14975, 300)
14975


,Median_BMI
DHSID,
AL200800000003,24.140174
AL200800000005,24.331148
AL200800000007,24.698591
AL200800000008,25.093687
AL200800000009,24.943476
...,...
ZW201500000382,23.111185
ZW201500000383,23.021541
ZW201500000386,24.600996


In [19]:
submission[label] = np.nan
submission.loc[eval.index, label] = output_df.loc[eval.index, label]

In [20]:
listt = [idx for idx in submission.index.tolist() if idx not in output_df.index.tolist()]

In [21]:
submission

,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate
DHSID,,,,,,
AL200800000003,24.936231,24.140174,50,50,50,50
AL200800000005,24.910430,24.331148,50,50,50,50
AL200800000007,25.094906,24.698591,50,50,50,50
AL200800000008,25.355694,25.093687,50,50,50,50
AL200800000009,25.350667,24.943476,50,50,50,50
...,...,...,...,...,...,...
ZW201500000382,23.931516,23.111185,50,50,50,50
ZW201500000383,24.330456,23.021541,50,50,50,50
ZW201500000386,24.926128,24.600996,50,50,50,50


## Under5_Mortality_Rate Model Inferencing
### Tuned at 2513 trees with validation loss = 5.417428231538735
#### 300 features, bin=255, cv=4 fold

In [22]:
df = pd.read_csv('/kaggle/input/under5-gee-new/U5MR_SHAP_Xtest_new.csv', index_col=0)
top_features = df['feature'][:300].tolist()

X = pd.read_parquet('/kaggle/input/under5-gee-new/train_data_Under5_df.parquet.gzip')[top_features]
y = pd.read_parquet('/kaggle/input/under5-gee-new/train_labels_Under5_df.parquet.gzip')
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=23)

# Convert data to LightGBM dataset format
params_d = {'max_bin': 255}
dtrain_data = lgb.Dataset(X, label=y, params=params_d)
# dtest_data = lgb.Dataset(X_valid, label=y_valid, params=params_d)

In [23]:
best_params = {'objective': 'regression',
 'metric': 'rmse',
 'verbosity': -1,
 'boosting': 'gbdt',
 'feature_pre_filter': False,
 'seed': 23,
 'num_threads': -1,
 'device_type': 'gpu',
 'num_gpu': 2,
 'extra_trees': True,
 'learning_rate': 0.013391730024885161,
 'num_leaves': 260,
 'feature_fraction': 0.6883368928797952,
 'feature_fraction_bynode': 0.917720715700967,
 'bagging_fraction': 0.9883929219308749,
 'bagging_freq': 101,
 'min_data_in_leaf': 8,
 'min_sum_hessian_in_leaf': 0.002272070249255062,
 'lambda_l1': 9.600755168316613,
 'lambda_l2': 0.2336692581654608,
 'min_gain_to_split': 1.7066341304215062}

In [24]:
verbose_eval=lgb.log_evaluation(period=10, show_stdv=True)
model = lgb.train(params=best_params, train_set=dtrain_data, num_boost_round=2513,
                      valid_sets=[dtrain_data],
                      valid_names=['training'],
                      callbacks=[verbose_eval])

[10]	training's rmse: 7.36568
[20]	training's rmse: 7.03276
[30]	training's rmse: 6.76879
[40]	training's rmse: 6.55246
[50]	training's rmse: 6.35238
[60]	training's rmse: 6.18907
[70]	training's rmse: 6.04827
[80]	training's rmse: 5.93243
[90]	training's rmse: 5.83218
[100]	training's rmse: 5.74837
[110]	training's rmse: 5.67291
[120]	training's rmse: 5.61214
[130]	training's rmse: 5.55861
[140]	training's rmse: 5.50882
[150]	training's rmse: 5.46145
[160]	training's rmse: 5.41983
[170]	training's rmse: 5.38617
[180]	training's rmse: 5.35455
[190]	training's rmse: 5.32337
[200]	training's rmse: 5.29362
[210]	training's rmse: 5.26796
[220]	training's rmse: 5.24467
[230]	training's rmse: 5.22099
[240]	training's rmse: 5.19938
[250]	training's rmse: 5.17853
[260]	training's rmse: 5.15946
[270]	training's rmse: 5.14058
[280]	training's rmse: 5.12358
[290]	training's rmse: 5.10743
[300]	training's rmse: 5.0921
[310]	training's rmse: 5.07813
[320]	training's rmse: 5.06337
[330]	training's r

In [25]:
# Save the trained model
model.save_model('Under5_Mortality_Rate.txt')

In [26]:
eval = pd.read_parquet('/kaggle/input/test-data-new/test_data_new.parquet.gzip')[top_features]
print(f'eval shape: {eval.shape}')
y_pred = model.predict(eval)
print(len(y_pred))
label = 'Under5_Mortality_Rate'
output_df = pd.DataFrame({'DHSID': eval.index, label: y_pred})
output_df.set_index('DHSID', inplace=True)
output_df

eval shape: (14975, 300)
14975


,Under5_Mortality_Rate
DHSID,
AL200800000003,5.592619
AL200800000005,5.523039
AL200800000007,4.927619
AL200800000008,4.984411
AL200800000009,4.460114
...,...
ZW201500000382,6.758557
ZW201500000383,5.691249
ZW201500000386,5.134293


In [27]:
submission[label] = np.nan
submission.loc[eval.index, label] = output_df.loc[eval.index, label]

In [28]:
listt = [idx for idx in submission.index.tolist() if idx not in output_df.index.tolist()]

In [29]:
submission

,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate
DHSID,,,,,,
AL200800000003,24.936231,24.140174,50,5.592619,50,50
AL200800000005,24.910430,24.331148,50,5.523039,50,50
AL200800000007,25.094906,24.698591,50,4.927619,50,50
AL200800000008,25.355694,25.093687,50,4.984411,50,50
AL200800000009,25.350667,24.943476,50,4.460114,50,50
...,...,...,...,...,...,...
ZW201500000382,23.931516,23.111185,50,6.758557,50,50
ZW201500000383,24.330456,23.021541,50,5.691249,50,50
ZW201500000386,24.926128,24.600996,50,5.134293,50,50


## Skilled_Birth_Attendant_Rate Model Inferencing
### Tuned at 2685 trees with validation loss = 19.337260216131018
#### 300 features, bin=255, cv=4 fold

In [30]:
df = pd.read_csv('/kaggle/input/skilled-birth/SBAR_SHAP_Xtest_new.csv', index_col=0)
top_features = df['feature'][:300].tolist()

X = pd.read_parquet('/kaggle/input/skilled-birth/train_data_Skilled_Birth_df.parquet.gzip')[top_features]
y = pd.read_parquet('/kaggle/input/skilled-birth/train_labels_Skilled_Birth_df.parquet.gzip')
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=23)

# Convert data to LightGBM dataset format
params_d = {'max_bin': 255}
dtrain_data = lgb.Dataset(X, label=y, params=params_d)
# dtest_data = lgb.Dataset(X_valid, label=y_valid, params=params_d)

In [31]:
best_params = {'objective': 'regression',
 'metric': 'rmse',
 'verbosity': -1,
 'boosting': 'gbdt',
 'feature_pre_filter': False,
 'seed': 23,
 'num_threads': -1,
 'device_type': 'gpu',
 'num_gpu': 2,
 'extra_trees': True,
 'learning_rate': 0.010465864103920423,
 'num_leaves': 318,
 'feature_fraction': 0.9438569721149388,
 'feature_fraction_bynode': 0.877051205867445,
 'bagging_fraction': 0.9544400479039109,
 'bagging_freq': 750,
 'min_data_in_leaf': 8,
 'min_sum_hessian_in_leaf': 0.0029734197140091796,
 'lambda_l1': 0.05653429511104152,
 'lambda_l2': 2.267907500788674,
 'min_gain_to_split': 4.132361339609545}

In [32]:
verbose_eval=lgb.log_evaluation(period=10, show_stdv=True)
model = lgb.train(params=best_params, train_set=dtrain_data, num_boost_round=2685,
                      valid_sets=[dtrain_data],
                      valid_names=['training'],
                      callbacks=[verbose_eval])

[10]	training's rmse: 33.6717
[20]	training's rmse: 31.9273
[30]	training's rmse: 30.4174
[40]	training's rmse: 29.0868
[50]	training's rmse: 27.8901
[60]	training's rmse: 26.8538
[70]	training's rmse: 25.917
[80]	training's rmse: 25.0807
[90]	training's rmse: 24.3596
[100]	training's rmse: 23.6926
[110]	training's rmse: 23.1251
[120]	training's rmse: 22.6071
[130]	training's rmse: 22.1512
[140]	training's rmse: 21.7386
[150]	training's rmse: 21.3554
[160]	training's rmse: 21.0304
[170]	training's rmse: 20.7202
[180]	training's rmse: 20.4421
[190]	training's rmse: 20.1807
[200]	training's rmse: 19.9522
[210]	training's rmse: 19.7406
[220]	training's rmse: 19.5405
[230]	training's rmse: 19.3607
[240]	training's rmse: 19.1884
[250]	training's rmse: 19.0285
[260]	training's rmse: 18.874
[270]	training's rmse: 18.7321
[280]	training's rmse: 18.5952
[290]	training's rmse: 18.4626
[300]	training's rmse: 18.335
[310]	training's rmse: 18.2158
[320]	training's rmse: 18.1076
[330]	training's rms

In [33]:
# Save the trained model
model.save_model('Skilled_Birth_Attendant_Rate.txt')

In [34]:
eval = pd.read_parquet('/kaggle/input/test-data-new/test_data_new.parquet.gzip')[top_features]
print(f'eval shape: {eval.shape}')
y_pred = model.predict(eval)
print(len(y_pred))
label = 'Skilled_Birth_Attendant_Rate'
output_df = pd.DataFrame({'DHSID': eval.index, label: y_pred})
output_df.set_index('DHSID', inplace=True)
output_df

eval shape: (14975, 300)
14975


,Skilled_Birth_Attendant_Rate
DHSID,
AL200800000003,88.566357
AL200800000005,84.167319
AL200800000007,79.214296
AL200800000008,69.238734
AL200800000009,73.119067
...,...
ZW201500000382,71.335657
ZW201500000383,87.448070
ZW201500000386,87.799098


In [35]:
submission[label] = np.nan
submission.loc[eval.index, label] = output_df.loc[eval.index, label]

In [36]:
listt = [idx for idx in submission.index.tolist() if idx not in output_df.index.tolist()]

In [37]:
submission

,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate
DHSID,,,,,,
AL200800000003,24.936231,24.140174,50,5.592619,88.566357,50
AL200800000005,24.910430,24.331148,50,5.523039,84.167319,50
AL200800000007,25.094906,24.698591,50,4.927619,79.214296,50
AL200800000008,25.355694,25.093687,50,4.984411,69.238734,50
AL200800000009,25.350667,24.943476,50,4.460114,73.119067,50
...,...,...,...,...,...,...
ZW201500000382,23.931516,23.111185,50,6.758557,71.335657,50
ZW201500000383,24.330456,23.021541,50,5.691249,87.448070,50
ZW201500000386,24.926128,24.600996,50,5.134293,87.799098,50


## Stunted_Rate Model Inferencing
### Tuned at 744 trees with validation loss = 18.926892957408526
#### 1500 features, bin=15, cv=3 fold

In [38]:
df = pd.read_csv('/kaggle/input/stunted-gee-new/SR_SHAP_Xtest_New.csv', index_col=0)
top_features = df['feature'][:1500].tolist()

X = pd.read_parquet('/kaggle/input/stunted-gee-new/train_data_Stunted_df.parquet.gzip')[top_features]
y = pd.read_parquet('/kaggle/input/stunted-gee-new/train_labels_Stunted_df.parquet.gzip')
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=23)

# Convert data to LightGBM dataset format
params_d = {'max_bin': 15}
dtrain_data = lgb.Dataset(X, label=y, params=params_d)
# dtest_data = lgb.Dataset(X_valid, label=y_valid, params=params_d)

In [39]:
best_params = {'objective': 'regression',
 'metric': 'rmse',
 'verbosity': -1,
 'boosting': 'gbdt',
 'feature_pre_filter': False,
 'seed': 23,
 'num_threads': -1,
 'device_type': 'gpu',
 'num_gpu': 2,
 'extra_trees': True,
 'learning_rate': 0.013461244294322886,
 'num_leaves': 123,
 'feature_fraction': 0.9492964393024804,
 'feature_fraction_bynode': 0.4195933941078986,
 'bagging_fraction': 0.9687382897635484,
 'bagging_freq': 79,
 'min_data_in_leaf': 100,
 'min_sum_hessian_in_leaf': 0.20527943565348275,
 'lambda_l1': 0.9209910224438882,
 'lambda_l2': 1.4624914034738177,
 'min_gain_to_split': 0.5036020167888964}

In [40]:
verbose_eval=lgb.log_evaluation(period=10, show_stdv=True)
model = lgb.train(params=best_params, train_set=dtrain_data, num_boost_round=744,
                      valid_sets=[dtrain_data],
                      valid_names=['training'],
                      callbacks=[verbose_eval])

[10]	training's rmse: 22.5744
[20]	training's rmse: 21.9273
[30]	training's rmse: 21.3854
[40]	training's rmse: 20.9264
[50]	training's rmse: 20.5402
[60]	training's rmse: 20.216
[70]	training's rmse: 19.9343
[80]	training's rmse: 19.6954
[90]	training's rmse: 19.4866
[100]	training's rmse: 19.3024
[110]	training's rmse: 19.1422
[120]	training's rmse: 18.9956
[130]	training's rmse: 18.8686
[140]	training's rmse: 18.7508
[150]	training's rmse: 18.6424
[160]	training's rmse: 18.5369
[170]	training's rmse: 18.4394
[180]	training's rmse: 18.3478
[190]	training's rmse: 18.2653
[200]	training's rmse: 18.1819
[210]	training's rmse: 18.1052
[220]	training's rmse: 18.0284
[230]	training's rmse: 17.9563
[240]	training's rmse: 17.8847
[250]	training's rmse: 17.8186
[260]	training's rmse: 17.7536
[270]	training's rmse: 17.6901
[280]	training's rmse: 17.6277
[290]	training's rmse: 17.5653
[300]	training's rmse: 17.5036
[310]	training's rmse: 17.4443
[320]	training's rmse: 17.3873
[330]	training's r

In [41]:
# Save the trained model
model.save_model('Stunted_Rate.txt')

In [42]:
eval = pd.read_parquet('/kaggle/input/test-data-new/test_data_new.parquet.gzip')[top_features]
print(f'eval shape: {eval.shape}')
y_pred = model.predict(eval)
print(len(y_pred))
label = 'Stunted_Rate'
output_df = pd.DataFrame({'DHSID': eval.index, label: y_pred})
output_df.set_index('DHSID', inplace=True)
output_df

eval shape: (14975, 1500)
14975


,Stunted_Rate
DHSID,
AL200800000003,13.263493
AL200800000005,13.166223
AL200800000007,11.167631
AL200800000008,11.464656
AL200800000009,11.293211
...,...
ZW201500000382,30.745452
ZW201500000383,24.809029
ZW201500000386,23.727862


In [43]:
submission[label] = np.nan
submission.loc[eval.index, label] = output_df.loc[eval.index, label]

In [44]:
listt = [idx for idx in submission.index.tolist() if idx not in output_df.index.tolist()]

In [45]:
submission

,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate
DHSID,,,,,,
AL200800000003,24.936231,24.140174,50,5.592619,88.566357,13.263493
AL200800000005,24.910430,24.331148,50,5.523039,84.167319,13.166223
AL200800000007,25.094906,24.698591,50,4.927619,79.214296,11.167631
AL200800000008,25.355694,25.093687,50,4.984411,69.238734,11.464656
AL200800000009,25.350667,24.943476,50,4.460114,73.119067,11.293211
...,...,...,...,...,...,...
ZW201500000382,23.931516,23.111185,50,6.758557,71.335657,30.745452
ZW201500000383,24.330456,23.021541,50,5.691249,87.448070,24.809029
ZW201500000386,24.926128,24.600996,50,5.134293,87.799098,23.727862


In [48]:
submission.to_csv('submission_wo_unmet.csv')

### Restarted the kernel and started again as the GPU RAM got exhausted

In [5]:
submission = pd.read_csv('/kaggle/input/test-data-new/submission_wo_unmet.csv',index_col=0)

In [6]:
submission

,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate
DHSID,,,,,,
AL200800000003,24.936231,24.140174,50,5.592619,88.566357,13.263493
AL200800000005,24.910430,24.331148,50,5.523039,84.167319,13.166223
AL200800000007,25.094906,24.698591,50,4.927619,79.214296,11.167631
AL200800000008,25.355694,25.093687,50,4.984411,69.238734,11.464656
AL200800000009,25.350667,24.943476,50,4.460114,73.119067,11.293211
...,...,...,...,...,...,...
ZW201500000382,23.931516,23.111185,50,6.758557,71.335657,30.745452
ZW201500000383,24.330456,23.021541,50,5.691249,87.448070,24.809029
ZW201500000386,24.926128,24.600996,50,5.134293,87.799098,23.727862


## Unmet_Need_Rate Model Inferencing
### Tuned at 1874 trees with validation loss = 18.873395172659954
#### 300 features, bin=255, cv=5 fold

In [7]:
df = pd.read_csv('/kaggle/input/unmet-gee-new/UNR_SHAP_Xtest_new.csv', index_col=0)
top_features = df['feature'][:300].tolist()

X = pd.read_parquet('/kaggle/input/unmet-gee-new/train_data_Unmet_df.parquet.gzip')[top_features]
y = pd.read_parquet('/kaggle/input/unmet-gee-new/train_labels_Unmet_df.parquet.gzip')
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=23)

# Convert data to LightGBM dataset format
params_d = {'max_bin': 255}
dtrain_data = lgb.Dataset(X, label=y, params=params_d)
# dtest_data = lgb.Dataset(X_valid, label=y_valid, params=params_d)

In [8]:
best_params = {'objective': 'regression',
 'metric': 'rmse',
 'verbosity': -1,
 'boosting': 'gbdt',
 'feature_pre_filter': False,
 'seed': 23,
 'num_threads': -1,
 'device_type': 'gpu',
 'num_gpu': 2,
 'extra_trees': True,
 'learning_rate': 0.01196039867562155,
 'num_leaves': 348,
 'feature_fraction': 0.9184098760658879,
 'feature_fraction_bynode': 0.9798322178480301,
 'bagging_fraction': 0.9887480231978242,
 'bagging_freq': 556,
 'min_data_in_leaf': 1,
 'min_sum_hessian_in_leaf': 0.9134912888597748,
 'lambda_l1': 5.584420640169413,
 'lambda_l2': 5.9225074896376855,
 'min_gain_to_split': 1.784299892453205}

In [9]:
verbose_eval=lgb.log_evaluation(period=10, show_stdv=True)
model = lgb.train(params=best_params, train_set=dtrain_data, num_boost_round=1874,
                      valid_sets=[dtrain_data],
                      valid_names=['training'],
                      callbacks=[verbose_eval])

[10]	training's rmse: 27.966
[20]	training's rmse: 26.6431
[30]	training's rmse: 25.498
[40]	training's rmse: 24.5254
[50]	training's rmse: 23.6962
[60]	training's rmse: 22.985
[70]	training's rmse: 22.3698
[80]	training's rmse: 21.8582
[90]	training's rmse: 21.4122
[100]	training's rmse: 21.0191
[110]	training's rmse: 20.6786
[120]	training's rmse: 20.3811
[130]	training's rmse: 20.1191
[140]	training's rmse: 19.8914
[150]	training's rmse: 19.6838
[160]	training's rmse: 19.4958
[170]	training's rmse: 19.3248
[180]	training's rmse: 19.1649
[190]	training's rmse: 19.0195
[200]	training's rmse: 18.8863
[210]	training's rmse: 18.7637
[220]	training's rmse: 18.6447
[230]	training's rmse: 18.5371
[240]	training's rmse: 18.4366
[250]	training's rmse: 18.3385
[260]	training's rmse: 18.2463
[270]	training's rmse: 18.1538
[280]	training's rmse: 18.0698
[290]	training's rmse: 17.9866
[300]	training's rmse: 17.9069
[310]	training's rmse: 17.8303
[320]	training's rmse: 17.7534
[330]	training's rms

In [10]:
# Save the trained model
model.save_model('Unmet_Need_Rate.txt')

In [11]:
eval = pd.read_parquet('/kaggle/input/test-data-new/test_data_new.parquet.gzip')[top_features]
print(f'eval shape: {eval.shape}')
y_pred = model.predict(eval)
print(len(y_pred))
label = 'Unmet_Need_Rate'
output_df = pd.DataFrame({'DHSID': eval.index, label: y_pred})
output_df.set_index('DHSID', inplace=True)
output_df

eval shape: (14975, 300)
14975


,Unmet_Need_Rate
DHSID,
AL200800000003,26.939070
AL200800000005,30.733021
AL200800000007,17.438489
AL200800000008,11.724036
AL200800000009,12.433995
...,...
ZW201500000382,17.469668
ZW201500000383,13.818759
ZW201500000386,29.165742


In [12]:
submission[label] = np.nan
submission.loc[eval.index, label] = output_df.loc[eval.index, label]

In [13]:
listt = [idx for idx in submission.index.tolist() if idx not in output_df.index.tolist()]

In [14]:
submission

,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate
DHSID,,,,,,
AL200800000003,24.936231,24.140174,26.939070,5.592619,88.566357,13.263493
AL200800000005,24.910430,24.331148,30.733021,5.523039,84.167319,13.166223
AL200800000007,25.094906,24.698591,17.438489,4.927619,79.214296,11.167631
AL200800000008,25.355694,25.093687,11.724036,4.984411,69.238734,11.464656
AL200800000009,25.350667,24.943476,12.433995,4.460114,73.119067,11.293211
...,...,...,...,...,...,...
ZW201500000382,23.931516,23.111185,17.469668,6.758557,71.335657,30.745452
ZW201500000383,24.330456,23.021541,13.818759,5.691249,87.448070,24.809029
ZW201500000386,24.926128,24.600996,29.165742,5.134293,87.799098,23.727862


In [15]:
submission.isnull().sum()

Mean_BMI                        25
Median_BMI                      25
Unmet_Need_Rate                 25
Under5_Mortality_Rate           25
Skilled_Birth_Attendant_Rate    25
Stunted_Rate                    25
dtype: int64

In [16]:
submission.describe()

,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate
count,14975.000000,14975.000000,14975.000000,14975.000000,14975.000000,14975.000000
mean,23.944267,23.464564,35.727987,8.079550,69.019977,29.499405
std,2.712117,2.687788,22.645419,4.426445,26.973905,13.523712
min,17.558916,18.023241,-2.887292,-0.156941,-3.575108,-1.011956
25%,22.017081,21.602642,16.554521,4.617148,51.034915,19.228553
50%,23.579775,22.967083,30.600274,7.638383,75.912572,29.410795
75%,25.241813,24.651658,51.717163,10.671273,92.597740,38.785231
max,33.883743,33.830235,105.535839,28.071541,106.087467,76.335151


## Post-Processing-1
### There are 25 missing DHSIDs in the real test data which have no features other than year of survey and country name. For such features we impute values or target variables based on rest inferenced data. First of all we group the submission data using parameters of country and year of survey. Then we impute median of respective group.

In [17]:
# Setting index as the first column
submission.reset_index(inplace=True)  # Reset index to create a new index column
# submission.rename(columns={'index': 'NewIndex'}, inplace=True)  # Rename the new index column if desired

In [18]:
submission['DHSCC'] = submission['DHSID'].apply(lambda x: x[:3] if x.startswith('DHS') else x[:2])
submission['DHSYEAR'] = submission['DHSID'].apply(lambda x: x[3:7] if x.startswith('DHS') else x[2:6])
submission['DHSCC'] = submission['DHSCC'].str.replace('DHS', 'BD')

submission.set_index('DHSID',inplace=True)

submission.loc[listt,'Mean_BMI':'Stunted_Rate']=np.nan
submission.isnull().sum()
# percentile_value1 = 40
# percentile_value2 = 60

grouped = submission.groupby(['DHSCC', 'DHSYEAR'])

def impute_median(series):
    return series.fillna(series.median())

def impute_percentile(series, percentile):
    return series.fillna(np.percentile(series.dropna(), percentile))

submission[['Mean_BMI','Median_BMI','Unmet_Need_Rate','Under5_Mortality_Rate','Skilled_Birth_Attendant_Rate','Stunted_Rate']] = grouped[['Mean_BMI','Median_BMI','Unmet_Need_Rate','Under5_Mortality_Rate','Skilled_Birth_Attendant_Rate','Stunted_Rate']].transform(impute_median)
# df['Under5_Mortality_Rate']=grouped['Under5_Mortality_Rate'].transform(lambda x: impute_percentile(x, percentile_value1))
# df['Skilled_Birth_Attendant_Rate']=grouped['Skilled_Birth_Attendant_Rate'].transform(lambda x: impute_percentile(x, percentile_value2))

submission.drop(['DHSYEAR','DHSCC'],axis=1,inplace=True)


In [19]:
submission.isnull().sum()

Mean_BMI                        0
Median_BMI                      0
Unmet_Need_Rate                 0
Under5_Mortality_Rate           0
Skilled_Birth_Attendant_Rate    0
Stunted_Rate                    0
dtype: int64

In [20]:
submission.describe()

,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate
count,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000
mean,23.953494,23.474081,35.694702,8.073261,69.051330,29.486392
std,2.722409,2.698891,22.641232,4.425878,26.970755,13.517958
min,17.558916,18.023241,-2.887292,-0.156941,-3.575108,-1.011956
25%,22.018268,21.604305,16.502494,4.604312,51.067048,19.183103
50%,23.584336,22.969958,30.546290,7.631426,75.954730,29.391919
75%,25.245139,24.656448,51.692588,10.664586,92.633229,38.769865
max,33.883743,33.830235,105.535839,28.071541,106.087467,76.335151


In [21]:
submission.loc['BO200800002008','Mean_BMI']= 23.655
submission.loc['BO200800002008','Median_BMI']= 21.815
submission.loc['BO200800002008','Unmet_Need_Rate']= 0.0
submission.loc['BO200800002008','Under5_Mortality_Rate']= 3.845
submission.loc['BO200800002008','Stunted_Rate']= 25.0
submission.loc['BO200800002008','Skilled_Birth_Attendant_Rate']= 100.0

## Post-Processing-2
### It is natural for Regressive Models to predict any real value. So it is possible that some values may cross over the physically possible limits. So there is clipping of values to bound the values within a particular specified range

In [22]:
import pandas as pd
import numpy as np

# Define the range for each column
column_ranges = {
    'Mean_BMI': (14.15, 55.48),
    'Median_BMI': (14.15, 55.48),
    'Unmet_Need_Rate': (0,100),
    'Under5_Mortality_Rate': (0,100),
    'Skilled_Birth_Attendant_Rate': (0,100),
    'Stunted_Rate': (0,100)
}

# Function to clip values and replace with the nearest value in the range
def clip_and_replace(column, column_range):
    lower_bound, upper_bound = column_range
    return np.clip(column, lower_bound, upper_bound)

# Apply the function to each column separately
for column, column_range in column_ranges.items():
    submission[column] = clip_and_replace(submission[column], column_range)

In [23]:
submission.describe()

,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate
count,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000
mean,23.953381,23.473898,35.691002,8.073124,69.018732,29.487146
std,2.722386,2.698910,22.632378,4.425928,26.927750,13.517208
min,17.558916,18.023241,0.000000,0.000000,0.000000,0.000000
25%,22.018268,21.604305,16.502494,4.604015,51.067048,19.184655
50%,23.584336,22.969135,30.546290,7.631426,75.954730,29.391919
75%,25.245079,24.656448,51.692588,10.664586,92.635410,38.769865
max,33.883743,33.830235,100.000000,28.071541,100.000000,76.335151


In [24]:
submission

,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate
DHSID,,,,,,
AL200800000003,24.936231,24.140174,26.939070,5.592619,88.566357,13.263493
AL200800000005,24.910430,24.331148,30.733021,5.523039,84.167319,13.166223
AL200800000007,25.094906,24.698591,17.438489,4.927619,79.214296,11.167631
AL200800000008,25.355694,25.093687,11.724036,4.984411,69.238734,11.464656
AL200800000009,25.350667,24.943476,12.433995,4.460114,73.119067,11.293211
...,...,...,...,...,...,...
ZW201500000382,23.931516,23.111185,17.469668,6.758557,71.335657,30.745452
ZW201500000383,24.330456,23.021541,13.818759,5.691249,87.448070,24.809029
ZW201500000386,24.926128,24.600996,29.165742,5.134293,87.799098,23.727862


In [25]:
submission.to_csv('final_submission.csv')